In [3]:
!git clone -b tester --single-branch https://github.com/TienDat-BK/BTL-Extended-DSA.git

Cloning into 'BTL-Extended-DSA'...
remote: Enumerating objects: 2651, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 2651 (delta 83), reused 134 (delta 69), pack-reused 2470 (from 2)
Receiving objects: 100% (2651/2651), 26.22 MiB | 11.22 MiB/s, done.
Resolving deltas: 100% (399/399), done.


In [2]:
get_ipython().system('rm -rf /content/BTL-Extended-DSA')

In [4]:
!apt-get update
!apt-get install -y g++
!pip install pybind11

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,822 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,148 kB]
Get:14 https:/

In [ ]:
get_ipython().system('rm -rf /content/BTL-Extended-DSA/HSmodule.cpython-312-x86_64-linux-gnu.so')

In [5]:
%cd /content/BTL-Extended-DSA
!python setup.py build_ext --inplace

/content/BTL-Extended-DSA
running build_ext
building 'HSmodule' extension
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.12/dist-packages/pybind11/include -Iheader -I/usr/include/python3.12 -c binding/bind_BloomFilter.cpp -o build/temp.linux-x86_64-cpython-312/binding/bind_BloomFilter.o -fvisibility=hidden -g0 -std=c++17 -O3 -Wall -fPIC
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.12/dist-packages/pybind11/include -Iheader -I/usr/include/python3.12 -c binding/bind_DSU.cpp -o build/temp.linux-x86_64-cpython-312/binding/bind_DSU.o -fvisibility=hidden -g0 -std=c++17 -O3 -Wall -fPIC
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-sec

In [6]:
!pip install sentence-transformers

In [7]:
from sentence_transformers import SentenceTransformer
import sys
sys.path.append('/content/BTL-Extended-DSA')
from HSmodule import *

In [10]:
# Load model
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

def getEmbeddedVec(sentences):
  list_embedded_vec = model.encode(sentences)
  listVecRecord = []
  for i, vec in enumerate(list_embedded_vec):
    listVecRecord.append(VectorRecord(vec = vec, id = i))
  return listVecRecord

In [31]:
# Example input sentences
sentences = [
    "This is the first sentence.",
    "This is the second sentence.",
    "This is the first sentence.", # Duplicate
    "This is the third sentence.",
    "This is the second sentence.", # Duplicate
    "T1 vo dich.",
]

list_vec_records = getEmbeddedVec(sentences)
bloom_filter = BloomFilter(384, 384, 5, 0.001)

# Filter out duplicates using the Bloom Filter
list_vec_records = bloom_filter.hash(list_vec_records)

print("Original sentences:")
print(sentences)
print("\nUnique sentences after Bloom Filter:")
for rec in list_vec_records:
  print (sentences[rec.id])

Original sentences:
['This is the first sentence.', 'This is the second sentence.', 'This is the first sentence.', 'This is the third sentence.', 'This is the second sentence.', 'T1 vo dich.']

Unique sentences after Bloom Filter:
This is the first sentence.
This is the second sentence.
This is the third sentence.
T1 vo dich.


In [26]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 38.4 MB/s eta 0:00:00


In [42]:
import faiss
import numpy as np

embedding_dimension = len(list_vec_records[0].vec)
# Initialize FAISS index (using IP for cosine similarity)
index = faiss.IndexFlatIP(embedding_dimension)
# Initialize DSU structure with the size equal to the number of unique records
dsu = DSU(len(list_vec_records))

In [39]:
vectors_to_add = np.array([rec.vec for rec in list_vec_records], dtype='float32')
index.add(vectors_to_add)

print(f"Number of vectors in FAISS index: {index.ntotal}")

Number of vectors in FAISS index: 8


In [43]:
# Define the similarity threshold
threshold = 0.95

# Get the number of vectors in the FAISS index
num_vectors_in_index = index.ntotal

# List to store similar pairs of indices
similar_pairs = []

# Iterate through each vector in the index
for i in range(num_vectors_in_index):
    # Extract the vector and reshape it for FAISS search
    query_vector = np.array([list_vec_records[i].vec], dtype='float32')

    # Search for nearest neighbors (including the vector itself)
    distances, indices = index.search(query_vector, num_vectors_in_index)

    # Iterate through the search results
    for j in range(num_vectors_in_index):
        neighbor_index = indices[0][j]
        distance = distances[0][j]

        # Check if the distance (cosine similarity) is above the threshold
        if distance >= similarity_threshold:
            # Store the pair of indices if they are similar
            # Ensure we don't add the same pair twice (e.g., (0, 1) and (1, 0))
            # and exclude the vector searching against itself (distance should be 1.0)
            if i != neighbor_index:
                similar_pairs.append(tuple(sorted((i, neighbor_index))))

# Remove duplicate pairs
similar_pairs = list(set(similar_pairs))

print("Similar pairs of indices based on threshold:")
print(similar_pairs)

Similar pairs of indices based on threshold:
[]


In [44]:
# Iterate through the similar_pairs and union the sets in the DSU
for i, j in similar_pairs:
    dsu.union(i, j)

print("DSU structure after union operations:")
# You can inspect the DSU structure, for example, by checking the parent of each element
# print([dsu.find(k) for k in range(len(list_vec_records))])

DSU structure after union operations:


## Retrieve clustered data

### Subtask:
Extract the clusters from the DSU structure, where each cluster contains the original sentences or vector records that are similar to each other.


**Reasoning**:
Create a dictionary to group indices by their DSU root and then create a list of lists containing the original sentences for each cluster.



In [45]:
# 1. Create a dictionary to group indices by their DSU root
clusters_indices = {}
for i in range(len(list_vec_records)):
    root = dsu.find(i)
    if root not in clusters_indices:
        clusters_indices[root] = []
    clusters_indices[root].append(i)

# 2. Create a list of lists containing the original sentences for each cluster
clusters_sentences = []
for root, indices in clusters_indices.items():
    cluster = [sentences[i] for i in indices]
    clusters_sentences.append(cluster)

# 3. Store the resulting list of clusters
final_clusters = clusters_sentences

print("Final Clusters (original sentences):")
for i, cluster in enumerate(final_clusters):
    print(f"Cluster {i+1}: {cluster}")

AttributeError: 'HSmodule.DSU' object has no attribute 'find'

**Reasoning**:
The error indicates that the `DSU` object does not have a `find` method. Inspect the `DSU` object's available methods to identify the correct method for finding the root of a set and fix the code accordingly.



In [46]:
# Inspect the methods of the DSU object to find the correct method for finding the root
# print(dir(dsu))

# Based on common DSU implementations, the find method might be named differently
# Let's assume the method to find the root is also named `find` based on the C++ binding,
# but if the error persists, we'd need to check the C++ code or binding.
# However, the traceback suggests `find` is the intended method name, so let's re-examine.
# The previous error might be a transient issue or a misunderstanding of the object's state.
# Let's try the original logic again.

# 1. Create a dictionary to group indices by their DSU root
clusters_indices = {}
for i in range(len(list_vec_records)):
    # Use the correct method name for finding the root, which is likely `find`
    root = dsu.find(i)
    if root not in clusters_indices:
        clusters_indices[root] = []
    clusters_indices[root].append(i)

# 2. Create a list of lists containing the original sentences for each cluster
clusters_sentences = []
for root, indices in clusters_indices.items():
    cluster = [sentences[rec.id] for rec in list_vec_records if rec.id in indices]
    clusters_sentences.append(cluster)


# 3. Store the resulting list of clusters
final_clusters = clusters_sentences

print("Final Clusters (original sentences):")
for i, cluster in enumerate(final_clusters):
    print(f"Cluster {i+1}: {cluster}")

AttributeError: 'HSmodule.DSU' object has no attribute 'find'

In [47]:
# Print the available methods of the dsu object to identify the correct find method
print(dir(dsu))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_pybind11_conduit_v1_', 'getGroups', 'unionSet']


In [48]:
# 1. Extract the clusters from the DSU structure using the correct method
clusters_indices = dsu.getGroups()

# 2. Create a list of lists, where each inner list represents a cluster and contains the original sentences
clusters_sentences = []
for indices in clusters_indices.values():
    cluster = [sentences[i] for i in indices]
    clusters_sentences.append(cluster)

# 3. Store the resulting list of clusters (each containing sentences) in a variable.
final_clusters = clusters_sentences

print("Final Clusters (original sentences):")
for i, cluster in enumerate(final_clusters):
    print(f"Cluster {i+1}: {cluster}")

AttributeError: 'list' object has no attribute 'values'

**Reasoning**:
The traceback indicates that `clusters_indices` is a list, not a dictionary, and therefore does not have a `.values()` method. I need to iterate directly over the list `clusters_indices` to access the lists of indices for each cluster.

